In [1]:
import os
import json
from datasets import load_dataset
from time import time
from dotenv import load_dotenv
from rich.console import Console
from psycopg import sql


/home/emmanuel/Documents/Work/Alba/Synced_projects/Alba_Waldo/waldo_postgresml/postgresml_alba/.devenv/state/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append('../pgml/')

from pgml import Database
from pgml.dbutils import run_select_statement


In [3]:
load_dotenv()
console = Console()

local_pgml = "postgres://postgres@127.0.0.1:5433/pgml"

conninfo = os.environ.get("PGML_CONNECTION", local_pgml)
db = Database(conninfo)


In [4]:
collection_name = "squad_collection"
collection = db.create_or_get_collection(collection_name)


In [5]:
data = load_dataset("squad", split="train")
data = data.to_pandas()
data = data.drop_duplicates(subset=["context"])

documents = [
    {"id": r["id"], "text": r["context"], "title": r["title"]}
    for r in data.to_dict(orient="records")
]


[16:46:07] WARNING  Found cached dataset squad                                                       ]8;id=451819;file:///home/emmanuel/Documents/Work/Alba/Synced_projects/Alba_Waldo/waldo_postgresml/postgresml_alba/.devenv/state/venv/lib/python3.10/site-packages/datasets/builder.py\builder.py]8;;\:]8;id=922399;file:///home/emmanuel/Documents/Work/Alba/Synced_projects/Alba_Waldo/waldo_postgresml/postgresml_alba/.devenv/state/venv/lib/python3.10/site-packages/datasets/builder.py#816\816]8;;\
                    (/home/emmanuel/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca4               
                    80ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)                                           

In [6]:
collection.upsert_documents(documents[:200])
collection.generate_chunks()


/home/emmanuel/Documents/Work/Alba/Synced_projects/Alba_Waldo/waldo_postgresml/postgresml_alba/.devenv/state/venv/l
ib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [7]:
collection.generate_embeddings()


Generating embeddings using intfloat/e5-small ...

In [8]:
start = time()
query = "Who won more than 20 grammy awards?"
results = collection.vector_search(query, top_k=5)
_end = time()


In [9]:
console.print("\nResults for '%s'" % (query), style="bold")
console.print(results)
console.print("Query time = %0.3f" % (_end - start))


Results for 'Who won more than 20 grammy awards?'

[
    {
        'score': 0.8538668523932447,
        'chunk': 'Beyoncé has won 20 Grammy Awards, both as a solo artist and member of Destiny\'s Child, making 
her the second most honored female artist by the Grammys, behind Alison Krauss and the most nominated woman in 
Grammy Award history with 52 nominations. "Single Ladies (Put a Ring on It)" won Song of the Year in 2010 while 
"Say My Name" and "Crazy in Love" had previously won Best R&B Song. Dangerously in Love, B\'Day and I Am... Sasha 
Fierce have all won Best Contemporary R&B Album. Beyoncé set the record for the most Grammy awards won by a female 
artist in one night in 2010 when she won six awards, breaking the tie she previously held with Alicia Keys, Norah 
Jones, Alison Krauss, and Amy Winehouse, with Adele equaling this in 2012. Following her role in Dreamgirls she was
nominated for Best Original Song for "Listen" and Best Actress at the Golden Globe Awards, and Outstanding Actress 
in a Motion Picture at the NAACP Image Awards. Beyoncé won two awards at the Broadcast Film Critics Association 
Awards 2006; Best Song for "Listen" and Best Original Soundtrack for Dreamgirls: Music from the Motion Picture.',
        'metadata': {'id': '2d06623c8b0024a03c5928439712844c', 'title': 'Beyoncé'}
    },
    {
        'score': 0.8404338692769873,
        'chunk': 'A self-described "modern-day feminist", Beyoncé creates songs that are often characterized by 
themes of love, relationships, and monogamy, as well as female sexuality and empowerment. On stage, her dynamic, 
highly choreographed performances have led to critics hailing her as one of the best entertainers in contemporary 
popular music. Throughout a career spanning 19 years, she has sold over 118 million records as a solo artist, and a
further 60 million with Destiny\'s Child, making her one of the best-selling music artists of all time. She has won
20 Grammy Awards and is the most nominated woman in the award\'s history. The Recording Industry Association of 
America recognized her as the Top Certified Artist in America during the 2000s decade. In 2009, Billboard named her
the Top Radio Songs Artist of the Decade, the Top Female Artist of the 2000s and their Artist of the Millennium in 
2011. Time listed her among the 100 most influential people in the world in 2013 and 2014. Forbes magazine also 
listed her as the most powerful female musician of 2015.',
        'metadata': {'id': 'f935aa1ff2467e9358949b1fb3d8e55c', 'title': 'Beyoncé'}
    },
    {
        'score': 0.8367417191064728,
        'chunk': 'At the 52nd Annual Grammy Awards, Beyoncé received ten nominations, including Album of the Year 
for I Am... Sasha Fierce, Record of the Year for "Halo", and Song of the Year for "Single Ladies (Put a Ring on 
It)", among others. She tied with Lauryn Hill for most Grammy nominations in a single year by a female artist. In 
2010, Beyoncé was featured on Lady Gaga\'s single "Telephone" and its music video. The song topped the US Pop Songs
chart, becoming the sixth number-one for both Beyoncé and Gaga, tying them with Mariah Carey for most number-ones 
since the Nielsen Top 40 airplay chart launched in 1992. "Telephone" received a Grammy Award nomination for Best 
Pop Collaboration with Vocals.',
        'metadata': {'id': '1df324ba685ceb7b1b7ff7e93767a731', 'title': 'Beyoncé'}
    },
    {
        'score': 0.8274824419662897,
        'chunk': 'Beyoncé has received numerous awards. As a solo artist she has sold over 15 million albums in the
US, and over 118 million records worldwide (a further 60 million additionally with Destiny\'s Child), making her 
one of the best-selling music artists of all time. The Recording Industry Association of America (RIAA) listed 
Beyoncé as the top certified artist of the 2000s, with a total of 64 certifications. Her songs "Crazy in Love", 
"Single Ladies (Put a Ring on It)", "Halo", and "Irreplaceable" are some of the best-selling singles of all time 
worldwide. In 2009, The Observe

Query time = 45.297

In [10]:
# Get the context passage and use pgml.transform to get short answer to the question
conn = db.pool.getconn()
context = " ".join(results[0]["chunk"].strip().split())
context = context.replace('"', '\\"').replace("'", "''")


In [11]:
select_statement = """SELECT pgml.transform(
    'question-answering',
    inputs => ARRAY[
        '{
            \"question\": \"%s\",
            \"context\": \"%s\"
        }'
    ]
) AS answer;""" % (
    query,
    context,
)


In [12]:
results = run_select_statement(conn, select_statement)
db.pool.putconn(conn)


In [13]:
console.print("\nResults for query '%s'" % query)
console.print(results)


Results for query 'Who won more than 20 grammy awards?'

[{'answer': {'end': 172, 'score': 0.4324144721031189, 'start': 159, 'answer': 'Alison Krauss'}}]

In [ ]:
db.archive_collection(collection_name)
